 - 损失函数可视化
 - 最优化
     - 有限差值法求梯度
     - 微分法求梯度：softmax为损失函数时推导

前面介绍了两部分：
 1. 评分函数：将原始图像像素映射为分类评分值
 2. 损失函数：根据评分函数和训练集图像数据实际分类的一致性，衡量某个具体参数集的质量好坏。
那么寻找到能使损失函数值最小化的参数的过程就是最优化 Optimization。

损失函数可视化
====
损失函数L可以看作是权重W的函数，在CIFAR-10中一个分类器的权重矩阵大小是[10,3073]，即$L(W_1,W_2,....,W10)$，对其中某一个分类器$W_i$有3073个参数，想要得到$L$关于$W_i$的可视化很难。
但是方法还是有的，随机生成一个权重矩阵W，并沿着此方向计算损失值，$L(W+\alpha W_1)$。

[具体参考这里！](https://zhuanlan.zhihu.com/p/21360434?refer=intelligentunit)
[还有这样一篇paper！](https://arxiv.org/abs/1712.09913)

总而言之，就是将高维空间压缩到二维，$W_i[1,3073]$转换到$[1,1]$然后在此基础上，画出loss关于它的值。
如果是压缩到三维，就是[1,3073]->[1,2]，那么完整的loss就是这个形状的3073/2*10维的版本。

最优化Optimization
==

在数学上我们已经知道loss下降最快的方向就是梯度方向（gradient）。

## 有限差值法计算梯度

公式：$\frac{df(x)}{dx} = \lim_{h\to 0}\frac{f(x+h)-f(x)}{h}$
下面代码是一个输入为函数f和向量x，计算f的梯度的通用函数，它返回函数f在点x处的梯度：

In [1]:
def eval_numerical_gradient(f, x):
  """  
  一个f在x处的数值梯度法的简单实现
  - f是只有一个参数的函数
  - x是计算梯度的点
  """ 

  fx = f(x) # 在原点计算函数值
  grad = np.zeros(x.shape)   ##
  h = 0.00001

  # 对x中所有的索引进行迭代
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:

    # 计算x+h处的函数值
    ix = it.multi_index
    old_value = x[ix]
    x[ix] = old_value + h # 增加h
    fxh = f(x) # 计算f(x + h)
    x[ix] = old_value # 存到前一个值中 (非常重要)

    # 计算偏导数
    grad[ix] = (fxh - fx) / h # 坡度
    it.iternext() # 到下个维度

  return grad

可以使用上面这个公式来计算任意函数在任意点上的梯度。下面计算权重空间中的某些随机点上，CIFAR-10损失函数的梯度：

In [4]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR_batch
from cs231n.classifiers.softmax import softmax_loss_vectorized

##原始数据
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py/data_batch_1' #只取一组数据
X_train, y_train = load_CIFAR_batch(cifar10_dir)  #(1000,32,32,3) (1000,1)
X_train = np.reshape(X_train, (X_train.shape[0], -1))   ##（1000,3072）
#归一化
mean_image = np.mean(X_train, axis = 0)
X_train -= mean_image
# add bias dimension and transform into columns
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])

##根据softmax损失函数据算loss,这个函数里面有用微分法计算梯度，但我们只取loss
def CIFAR_loss_fun(W):
    loss,dw = softmax_loss_vectorized(W, X_train, y_train, 0.000005)
    return  loss

W = np.random.randn(3073, 10) * 0.0001  ##随机权重向量
df = eval_numerical_gradient(CIFAR_loss_fun, W) ##计算权重空间下任意点关于loss的梯度

loss_original = CIFAR_loss_fun(W)  ##初始损失值
print("original loss: %f"%(loss_original,))

#查看不同步长的效果
for step_size_log in [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]:
    step_size = 10**step_size_log
    W_new = W-step_size*df
    loss_new = CIFAR_loss_fun(W_new)
    print("for step size %f new loss: %f" % (step_size, loss_new))
    

original loss: 2.379955
for step size 0.000000 new loss: 2.379945
for step size 0.000000 new loss: 2.379855
for step size 0.000000 new loss: 2.378953
for step size 0.000000 new loss: 2.370042
for step size 0.000001 new loss: 2.291368
for step size 0.000010 new loss: 2.294673
for step size 0.000100 new loss: 12.200434


/home/panxie/文档/cs231n/assignment1/cs231n/classifiers/softmax.py:85: RuntimeWarning: divide by zero encountered in log
  loss = np.sum(-np.log(coef[range(num_train), y]))


for step size 0.001000 new loss: inf
for step size 0.010000 new loss: inf
for step size 0.100000 new loss: inf


## 微分分析计算梯度

有限差值法太太太太慢了，而且终究只是近似。第二个梯度计算方法是利用微分来分析，能得到计算梯度的公式（不是近似），用公式计算梯度速度很快，唯一不好的就是实现的时候容易出错。为了解决这个问题，在实际操作时常常将分析梯度法的结果和数值梯度法的结果作比较，以此来检查其实现的正确性，这个步骤叫做梯度检查。

以sofemax为例进行推导：

对单个样本其损失值：
$L_i = -\log(\frac{e^{f_{y_i}}}{\sum_je^{f_j}})$

整个数据集的损失值：
$L = \frac{1}{N}\sum_iL_i +\lambda \sum_k\sum_lW_{k,l}^2 $

$L = \frac{1}{N}\sum_i-\log(\frac{e^{f_{y_i}}}{\sum_je^{f_j}}) +\lambda \sum_k\sum_lW_{k,l}^2$    
这里$f_{y_i}$是一个样本对应的真实标签的值，shape=(1,)

$L = \frac{1}{N}\sum_i-\log(\frac{e^{f_{y_i}}}{\sum_je^{f_j}}) +\lambda \sum_k\sum_lW_{k,l}^2
= \frac{1}{N}\sum_i(log(\sum_je^{f_j})-f_{y_i})+\lambda \sum_k\sum_lW_{k,l}^2$   
这里$f_{y_i}$是所有样本对应的真实标签的值,shape=(N,1) 代码中： score[range(N),y]


其中$f_{y_i}=W_{y_i}x$, $f_{_i}=W_{j \ne y_i}x$ 

对W求导，$W_{y_i}$和$W_{j\ne y_i}$的求导是不一样的

$\dfrac{\partial L}{\partial W_{y_i}} = \dfrac{1}{N}(\sum_i\dfrac{e^{W_{y_i}x}}{\sum_je^{W_jx}}x-xI(j==y_i))+2\lambda W$  
#### 注意这其中的求和，先以行为单位求和，即每个样本的loss，然后以列为单位求和，求出总的loss

代码实现：

In [ ]:
score = X.dot(W)  ##(N,3073)*(3073,10) = (N,10)
score -= np.max(score, axis=1, keepdims=True)  # [N,10]
exp_score = np.exp(score)  # [N,10]
sum_score = np.sum(exp_score, axis=1, keepdims=True)   ##(N,1)以行为单位求和
coef = exp_score / sum_score #(N,10)
loss = np.sum(-np.log(coef[range(num_train), y])) ##取出coef中每个样本真实标签对应的那列，然后以行为单位求和，即总loss
loss /= num_train
loss += reg * np.sum(W * W)

coef_yi = np.zeros_like(coef)
coef_yi[range(num_train), y] = 1  ##W矩阵中{j==y_i}每一行真实标签对应的位置参数要-1
dW = X.T.dot(coef - coef_yi)
dW /= num_train
dW += reg * 2 * W

以多类SVM为例求梯度：

In [ ]:
#求loss   y = max(0,x_j-x_{y_j}+1)   margin = x_j-x_{y_j}+1
N = x.shape[0]
correct_class_scores = x[np.arange(N), y]   ## 这得到的是一个行向量(N,1)
margins = np.maximum(0, x - correct_class_scores[:, np.newaxis] + 1.0)   #[:, np.newaxis]将行向量转换为列向量
margins[np.arange(N), y] = 0   # 真实标签对应的列是1,要转换为0
loss = np.sum(margins) / N
#求梯度   
num_pos = np.sum(margins > 0, axis=1)  ## 每行大于0的个数
dx = np.zeros_like(x)
dx[margins > 0] = 1    
dx[np.arange(N), y] -= num_pos  # ha？ 哦哦，W_{y_j}在margin大于0是求导为-1,所以每一行对应的是这一行大于0的个数 -pos
dx /= N
return loss, dx